**Setting Up Google Colab and Mounting Google Drive**



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Extracting .rar Files in Colab**

1.   Install the unrar tool to handle .rar files in Colab.
2.   Extract the dataset. Move your .rar files to your Google Drive and specify the file path to extract them. Replace "your_dataset.rar" with the actual filename in your Drive.

This will extract the files into a directory named /content/dataset/. Now, you should have subfolders like COVID-19, Pneumonia, and Normal, depending on the dataset structure.



In [2]:
# Install unrar to handle .rar files
!apt-get install unrar


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


**Flatten the Directory Structure After Extraction**

We’ll modify the extraction process to automatically move images from the nested folders to a single-level structure. Here’s how you can do it:

In [5]:
import os
import shutil

# Paths to the .rar files
covid_rar = '/content/drive/MyDrive/PROJECT_HDA/data/covid.rar'
normal_rar = '/content/drive/MyDrive/PROJECT_HDA/data/normal.rar'
pneumonia_rar = '/content/drive/MyDrive/PROJECT_HDA/data/pneumonia.rar'

# Define the target directories for each class
target_dirs = {
    'covid': '/content/dataset/covid',
    'normal': '/content/dataset/normal',
    'pneumonia': '/content/dataset/pneumonia'
}

# Ensure target directories exist
for dir_path in target_dirs.values():
    os.makedirs(dir_path, exist_ok=True)

# Step 1: Extract each .rar file into a temporary directory
temp_extract_dir = '/content/temp_extract/'  # Temporary directory for nested extraction
os.makedirs(temp_extract_dir, exist_ok=True)

# Extract and flatten each .rar file
!unrar x {covid_rar} {temp_extract_dir}
!unrar x {normal_rar} {temp_extract_dir}
!unrar x {pneumonia_rar} {temp_extract_dir}

# Step 2: Move images from nested folders to the correct target directories
for cls, target_dir in target_dirs.items():
    cls_temp_dir = os.path.join(temp_extract_dir, cls)  # Navigate to nested directory

    # Walk through all subdirectories and find image files
    for root, _, files in os.walk(cls_temp_dir):
        for file in files:
            # Only move image files (e.g., .jpg, .jpeg, .png)
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                # Move each image to the target directory for its class
                shutil.move(os.path.join(root, file), os.path.join(target_dir, file))

print("Extraction and directory flattening complete.")



UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/PROJECT_HDA/data/covid.rar

Creating    /content/temp_extract/covid                               OK
Extracting  /content/temp_extract/covid/01E392EE-69F9-4E33-BFCE-E5C968654078-768x572.jpeg       0%  OK 
Extracting  /content/temp_extract/covid/03BF7561-A9BA-4C3C-B8A0-D3E585F73F3C-1068x1083.jpeg       0%  OK 
Extracting  /content/temp_extract/covid/0C7E78DA-FAFC-480D-88B6-1459C51481AF-1068x817.jpeg       0%  OK 
Extracting  /content/temp_extract/covid/1312A392-67A3-4EBF-9319-810CF6DA5EF6-1068x1086.jpeg       0%  OK 
Extracting  /content/temp_extract/covid/134895D3-77B8-4546-B7E5-C7DABD600309-1068x911.jpeg       0%  OK 
Extracting  /content/temp_extract/covid/1B734A89-A1BF-49A8-A1D3-66FAFA4FAC5D-768x635.jpeg       0%  OK 
Extracting  /content/temp_extract/covid/1F6343EE-AFEC-4B7D-97F5-62797EE18767-1068x1064.jpeg     

In [6]:
import os

# List the extracted files in each folder
print("COVID files:", os.listdir('/content/dataset/covid')[:5])  # Show first 5 files
print("Normal files:", os.listdir('/content/dataset/normal')[:5])
print("Pneumonia files:", os.listdir('/content/dataset/pneumonia')[:5])


COVID files: ['COVID-19 (390).jpg', 'COVID-19 (451).jpg', 'COVID-19 (624).jpg', 'COVID19(165).jpg', 'COVID-19 (704).jpg']
Normal files: ['00000840_000.png', '00000947_001.png', '00000652_008.png', '00000393_000.png', '00000054_008.png']
Pneumonia files: ['person1261_virus_2148.jpeg', 'person357_bacteria_1639.jpeg', 'person276_bacteria_1298.jpeg', 'person1865_bacteria_4737.jpeg', 'person1349_bacteria_3437.jpeg']


**Environment Setup and Dataset Preparation**

Set up the basic imports for model building, data loading, and visualization.

In [7]:
#required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

**Create Train/Test Split**

We’ll create a train and test split programmatically:

Set up separate train and test folders, create a structure for the train and test sets, with 80% of the images in train and 20% in test.

Copy a portion of images from each class folder into the new train and test folders.We’ll use shutil to copy files into new directories.

After running this code, your directory structure should look like this:

/content/dataset_split/

├── train/
│   ├── covid/
│   ├── normal/
│   └── pneumonia/

└── test/
    ├── covid/
    ├── normal/
    └── pneumonia/

In [8]:
import os
import random
import shutil

# # Define paths for the train and test split
data_dir = "/content/dataset"
train_dir = "/content/dataset_split/train"
test_dir = "/content/dataset_split/test"

# Define the class names
classes = ['covid', 'normal', 'pneumonia']

# Define the split ratio
train_ratio = 0.8

# Create train and test directories for each class/ Loop over each class folder to split images
for cls in classes:
# Create the class directories in train and test folders
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

# Source folder for each class
    cls_folder = os.path.join(data_dir, cls)
#    cls_folder = f"/content/dataset/{cls}"
    print(f"Contents of {cls_folder}:")
    print(os.listdir(cls_folder))

    # Get list of images in each class folder
    images = [img for img in os.listdir(cls_folder) if os.path.isfile(os.path.join(cls_folder, img))]

    # Shuffle images and split based on train_ratio
    random.shuffle(images)
    split_idx = int(len(images) * train_ratio)
    train_images = images[:split_idx]
    test_images = images[split_idx:]

    # Copy images to the respective train and test directories
    for img in train_images:
        shutil.copy(os.path.join(cls_folder, img), os.path.join(train_dir, cls, img))
    for img in test_images:
        shutil.copy(os.path.join(cls_folder, img), os.path.join(test_dir, cls, img))

print("Dataset split and organization complete.")

Contents of /content/dataset/covid:
['COVID-19 (390).jpg', 'COVID-19 (451).jpg', 'COVID-19 (624).jpg', 'COVID19(165).jpg', 'COVID-19 (704).jpg', 'COVID19(396).jpg', 'COVID-19 (407).jpg', 'COVID-19 (90).jpg', 'COVID-19 (343).jpg', 'COVID-19 (806).jpg', 'COVID-19 (274).jpg', 'COVID19(203).jpg', 'COVID-19 (113).jpg', 'COVID19(398).jpg', 'COVID19(271).jpg', 'COVID-19 (22).jpeg', 'COVID-19 (483).jpg', 'COVID19(76).jpg', 'COVID-19 (458).jpg', 'COVID-19 (277).jpg', 'COVID-19 (841).jpg', 'COVID-19 (584).jpg', 'COVID19(312).jpg', 'COVID19(178).jpg', 'COVID19(307).jpg', 'COVID-19 (506).jpg', 'COVID19(531).jpg', 'COVID-19 (774).jpg', 'COVID19(542).jpg', 'COVID-19 (233).jpg', 'COVID-19 (317).jpg', 'COVID19(373).jpg', 'COVID-19 (1).jpg', 'COVID19(453).jpg', 'COVID-19 (249).jpg', 'COVID19(162).jpg', 'COVID-19 (513).jpg', 'COVID-19 (44).jpg', 'COVID-19 (718).jpg', 'COVID-19 (644).jpg', 'COVID19(314).jpg', '7AF6C1AF-D249-4BD2-8C26-449304105D03-1068x881.jpeg', 'COVID19(156).jpg', 'COVID19(196).jpg', 'C

**Verify Image Paths and Formats**

Check if the images were copied correctly into the /content/dataset_split/train and /content/dataset_split/test folders.

In [9]:
import os

# Check if images are correctly copied and have valid formats
train_dir = "/content/dataset_split/train"
test_dir = "/content/dataset_split/test"
valid_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}

# Function to list files in a directory and check extensions
def check_images(directory):
    for cls in os.listdir(directory):
        class_dir = os.path.join(directory, cls)
        if os.path.isdir(class_dir):
            print(f"Checking '{cls}' class in '{directory}':")
            images = [f for f in os.listdir(class_dir) if os.path.splitext(f)[1].lower() in valid_extensions]
            print(f"  Found {len(images)} valid images.")
            if len(images) == 0:
                print("  Warning: No valid images found in this class folder!")

# Check train and test directories
check_images(train_dir)
check_images(test_dir)


Checking 'pneumonia' class in '/content/dataset_split/train':
  Found 1220 valid images.
Checking 'covid' class in '/content/dataset_split/train':
  Found 1220 valid images.
Checking 'normal' class in '/content/dataset_split/train':
  Found 1220 valid images.
Checking 'pneumonia' class in '/content/dataset_split/test':
  Found 305 valid images.
Checking 'covid' class in '/content/dataset_split/test':
  Found 305 valid images.
Checking 'normal' class in '/content/dataset_split/test':
  Found 305 valid images.


**Define Dataset and Transformations**



**Setting Up Data Loaders with PyTorch**

Once the dataset is extracted, we can set up PyTorch data loaders.


1.   Define data transformations for training and testing datasets.
2.   Set up the ImageFolder dataset and DataLoader for easy batching and shuffling.

With the dataset organized, we can set up the data loaders for training and testing. The ImageFolder class will now recognize the directory structure and automatically assign labels based on folder names.

This setup will allow your model to use the data in batches, with transformations applied during loading for augmentation and normalization.


In [14]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations for the training set with data augmentation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),                 # Resize images to 224x224
    transforms.RandomHorizontalFlip(),             # Apply horizontal flip randomly
    transforms.RandomRotation(10),                 # Apply random rotation up to 10 degrees
    transforms.ToTensor(),                         # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize with ImageNet mean and std
])

# Define transformations for the test set without augmentation
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),                 # Resize images to 224x224
    transforms.ToTensor(),                         # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize with ImageNet mean and std
])

# Load the train and test datasets with ImageFolder
train_data = datasets.ImageFolder("/content/dataset_split/train", transform=train_transform)
test_data = datasets.ImageFolder("/content/dataset_split/test", transform=test_transform)
# Test Command for checking comit works.
# Define data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


FileNotFoundError: Found no valid file for the classes covid, normal, pneumonia. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [11]:
#checking the the git repo

Cloning into 'lung-disease-prediction'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (6/6), 30.66 KiB | 1.46 MiB/s, done.
